In [1]:
import os
import librosa
from IPython.display import Audio
os.environ['CUDA_VISIBLE_DEVICES'] = '1' 
import torch

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
guitar_path = '/speech/dbwork/mul/spielwiese3/deuhlics/tency_writeout_20230710/tency1_10instr_dblbass2uprbass/part1/acdc_hells-bells_13787/guitar.flac'
bass_path = '/speech/dbwork/mul/spielwiese3/deuhlics/tency_writeout_20230710/tency1_10instr_dblbass2uprbass/part1/alice-cooper_poison_13025/bass.flac'
drums_path = '/speech/dbwork/mul/spielwiese3/deuhlics/tency_writeout_20230710/tency1_10instr_dblbass2uprbass/part1/alice-cooper_poison_13025/drums.flac'
vocal_path = '/speech/dbwork/mul/spielwiese3/deuhlics/tency_writeout_20230710/tency1_10instr_dblbass2uprbass/part1/alice-cooper_poison_13025/vocals.flac'
other_path = '/speech/dbwork/mul/spielwiese3/deuhlics/tency_writeout_20230710/tency1_10instr_dblbass2uprbass/part1/alice-cooper_poison_13025/other.flac'

In [4]:
# Target sampling rate
target_sr = 16000

# Parameters for the segment you want to load
start_time =60      # Start time in seconds (adjust as needed)
duration = 5.0      # Duration in seconds

# Load guitar
guitar_data, _ = librosa.load(
    guitar_path,
    sr=target_sr,
    offset=start_time,
    duration=duration,
    mono=True
)

# Load bass
bass_data, _ = librosa.load(
    bass_path,
    sr=target_sr,
    offset=start_time,
    duration=duration,
    mono=True
)

# Load drums
drums_data, _ = librosa.load(
    drums_path,
    sr=target_sr,
    offset=start_time,
    duration=duration,
    mono=True
)

# Load vocal
vocal_data, _ = librosa.load(
    vocal_path,
    sr=target_sr,
    offset=start_time,
    duration=duration,
    mono=True
)

# Load other
other_data, _ = librosa.load(
    other_path,
    sr=target_sr,
    offset=start_time,
    duration=duration,
    mono=True
)

# Ensure all audio arrays are the same length
min_length = min(len(guitar_data), len(bass_data), len(drums_data))
guitar_data = guitar_data[:min_length]
bass_data = bass_data[:min_length]
drums_data = drums_data[:min_length]
vocal_data = vocal_data[:min_length]
other_data = other_data[:min_length]

# Sum the audio data to mix the tracks
mix =  guitar_data + bass_data + drums_data + vocal_data + other_data
stem = guitar_data



In [5]:
# Play the guitar stem
print("Guitar Stem:")
Audio(data=guitar_data, rate=target_sr)

Guitar Stem:


In [6]:
# Play the bass stem
print("Bass Stem:")
Audio(data=bass_data, rate=target_sr)

Bass Stem:


In [7]:
# Play the drums stem
print("Drums Stem:")
Audio(data=drums_data, rate=target_sr)

Drums Stem:


In [8]:
# Play the vocal stem
print("Vocal Stem:")
Audio(data=vocal_data, rate=target_sr)


Vocal Stem:


In [9]:
# Play the other stem
print("Other Stem:")
Audio(data=other_data, rate=target_sr)

Other Stem:


In [10]:
# Play the mixed audio
print("Mixed Audio:")
Audio(data=mix, rate=target_sr)

Mixed Audio:


In [11]:
from contrastive_model import constants
from contrastive_model.contrastive_model import CoCola
from feature_extraction.feature_extraction import CoColaFeatureExtractor


model = CoCola.load_from_checkpoint("/home/demancum/cocola_test/checkpoint-epoch=119-val_loss=0.00.ckpt")
model.to(device)
feature_extractor = CoColaFeatureExtractor()
feature_extractor.to(device)

model.eval()

# Set to:
# - constants.EmbeddingMode.BOTH for standard COCOLA Score
# - constants.EmbeddingMode.HARMONIC for COCOLA Harmonic Score
# - constants.EmbeddingMode.PERCUSSIVE for COCOLA Percussive Score
model.set_embedding_mode(constants.EmbeddingMode.BOTH)

mix_tensor = torch.tensor(mix).to(device)
stem_tensor = torch.tensor(stem).to(device)

features_x = feature_extractor(mix_tensor.unsqueeze(0))
features_y = feature_extractor(stem_tensor.unsqueeze(0))

features_x = features_x.to(device)
features_y = features_y.to(device)

score = model.score(features_x, features_y)

In [12]:
score

tensor(22.7232, device='cuda:0', grad_fn=<SqueezeBackward1>)

In [4]:
import torchaudio.transforms as T

In [5]:
feature_extractor = torch.nn.Sequential(
                T.MelSpectrogram(
                    sample_rate=16000,
                    n_fft=400,
                    #win_length=self.win_length,
                    #hop_length=self.hop_length,
                    #f_min=self.f_min,
                    #f_max=self.f_max,
                    #n_mels=self.n_mels,
                ),
                T.AmplitudeToDB()
)

/home/demancum/miniconda3/envs/cocola_test/lib/python3.11/site-packages/torchaudio/functional/functional.py:584: UserWarning: At least one mel filterbank has all zero values. The value for `n_mels` (128) may be set too high. Or, the value for `n_freqs` (201) may be set too low.
  warnings.warn(


In [10]:
x = torch.randn(2, 80000)

In [11]:
x.shape

torch.Size([2, 80000])

In [12]:
y = feature_extractor(x)
y.shape

torch.Size([2, 128, 401])